## 准备数据

In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [12]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [13]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 256], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(tf.random.normal([256, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])          # 展平输入 [batch_size, 784]
        h = tf.nn.relu(x @ self.W1 + self.b1) # 隐藏层计算 [batch_size, 256]
        logits = h @ self.W2 + self.b2        # 输出层计算 [batch_size, 10]
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [14]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [16]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.6842513 ; accuracy 0.4572
epoch 1 : loss 1.6735117 ; accuracy 0.46348333
epoch 2 : loss 1.6629058 ; accuracy 0.4701
epoch 3 : loss 1.6524328 ; accuracy 0.47633332
epoch 4 : loss 1.6420909 ; accuracy 0.48211667
epoch 5 : loss 1.6318777 ; accuracy 0.48763335
epoch 6 : loss 1.6217914 ; accuracy 0.49296665
epoch 7 : loss 1.6118313 ; accuracy 0.49866667
epoch 8 : loss 1.6019956 ; accuracy 0.50456667
epoch 9 : loss 1.5922818 ; accuracy 0.5097333
epoch 10 : loss 1.5826886 ; accuracy 0.5150333
epoch 11 : loss 1.5732133 ; accuracy 0.52085
epoch 12 : loss 1.5638554 ; accuracy 0.52598333
epoch 13 : loss 1.554614 ; accuracy 0.53105
epoch 14 : loss 1.5454873 ; accuracy 0.5364
epoch 15 : loss 1.5364742 ; accuracy 0.5417167
epoch 16 : loss 1.5275725 ; accuracy 0.5467333
epoch 17 : loss 1.51878 ; accuracy 0.5517833
epoch 18 : loss 1.5100949 ; accuracy 0.55625
epoch 19 : loss 1.5015162 ; accuracy 0.56065
epoch 20 : loss 1.4930421 ; accuracy 0.56493336
epoch 21 : loss 1.4846718 ; accura